In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
import tensorflow as tf
from keras.metrics import Precision, Recall
from keras.layers import Reshape, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, Concatenate
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
from PIL import Image
import numpy as np
import os

def load_png_images_from_folder(folder_path, target_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            label = int(filename[0])  # Assuming the label is the first character in the filename
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img = img.resize(target_size)  # Resize the image to a consistent size
            images.append(np.array(img))
            labels.append(label)
    return np.array(images), np.array(labels)

# Replace 'path/to/your/images' with the actual path to your folder containing .png files
image_folder_path = 'img/urdu_numerals/'
images, labels = load_png_images_from_folder(image_folder_path)

# Print the shape of the created dataset
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

In [ ]:
np.savez("datasets/urdu_numericals.npz", image=images, label = labels)

In [ ]:
model =tf.keras.models.load_model(
        "models/numerical_model_convolutional.h5"
    ) 

model.summary()

In [ ]:
labels_x = to_categorical(labels)

In [ ]:
for m in model.layers:
    print(m.name)
new_model = Sequential()


In [ ]:
new_model.summary()

In [ ]:
new_model = Sequential()
new_model.add(Conv2D(32, (2, 2), activation='relu', input_shape=(224, 224,1)))
new_model.add(MaxPooling2D(pool_size=(2, 2), name='max_pooling2d_newmodel'))   # Changed (3, 3) to (2, 2)

for idx, layer in enumerate(model.layers):
    if idx not in [0,5,6,7]:
        # Append the index to the layer name to make it unique
        new_layer_name = f"{layer.name}_new_{idx}"
        layer._name = new_layer_name
        layer.trainable = False
        new_model.add(layer)

new_model.add(Conv2D(32, (1, 1), activation='relu',name="Conv2D_2_new"))
new_model.add(MaxPooling2D(1, 1,name="maxpo0ling_new"))
new_model.add(Flatten(name="flatten_new"))
new_model.add(Dense(64, activation='softmax',name="dense_final_new"))
new_model.add(Dense(10, activation='softmax', name="dense_final_new_new"))

new_model.summary()


In [ ]:
for layer in new_model.layers:
    print(layer.trainable)

In [ ]:
for layer in new_model.layers:
    print(layer)
var1 = new_model.layers[0].input

# new_model = tf.keras.Model(inputs=var1, outputs=final_layer)
#new_model.build()
new_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
new_model.layers[-1]

In [ ]:
new_model.summary()

In [ ]:
new_model.fit(images,labels_x, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
images.shape

In [ ]:
new_model.save("models/numerical_model_convolutional_2_fine_tuned_urdu_224.h5")